### Character-level Sequence-to-Sequence for Machine Translation - [Keras tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

implement a basic character-level sequence-to-sequence model to translate short English sentences into short French sentences, character-by-character. Note that it is fairly unusual to do character-level, rather than word-level machine translation.

In [1]:
# Use a dataset of pairs of English sentences and their French translation

!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip -d fra-eng
!more fra-eng/fra.txt

--2019-07-19 04:02:11--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3467257 (3.3M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   3.31M  2.15MB/s    in 1.5s    

2019-07-19 04:02:13 (2.15 MB/s) - ‘fra-eng.zip’ saved [3467257/3467257]

Archive:  fra-eng.zip
  inflating: fra-eng/_about.txt      
  inflating: fra-eng/fra.txt         
Go.	Va !
Hi.	Salut !
Hi.	Salut.
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Stop!	Ça suffit !
Stop!	Stop !
Stop!	Arrête-toi !
Wait!	Attends !
Wait!	Attendez !
Go on.	Poursuis.
Go on.	Continuez.
Go on.	Poursuivez.
Hello!	Bonjour !
Hello!	Salut !
I see.	Je comprends.
I try.	J'essaye.
I won!	J'ai gagné !


A few samples from the dataset

Stop!	Arrête-toi !

Wait!	Attends !

Wait!	Attendez !

Go on.	Poursuis.

Go on.	Continuez.

Go on.	Poursuivez.

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [0]:
def word2char (line):
   char = [c for c in line]
   return char 

def max_sentence_length (char_sentences):
  max_sentence_length = 0
  for char_sentence in char_sentences:
    max_sentence_length = max (max_sentence_length, len (char_sentence))
  return (max_sentence_length)    

eng_char_sentences = []
fr_char_sentences = []
fr_char_sentences_shift = []
with open(data_path, "r") as fp:
  # Read the file line by line
  line = fp.readline()
  samples = 0
  while line and (samples < num_samples):
    # Split the line as <eng_sentence>TAB<fr_sentence>
    eng_sentence,fr_sentence = line.split('\t')
    fr_sentence_shift = fr_sentence
    fr_sentence = '\t' + fr_sentence
    
    # Convert the sentence from a string to an array of characters
    eng_char_sentence = word2char (eng_sentence)
    fr_char_sentence = word2char (fr_sentence)
    fr_char_sentence_shift = word2char (fr_sentence_shift)
    
    # Add to the array of sentences, each sentence in turn being an array of characters
    eng_char_sentences.append (eng_char_sentence)
    fr_char_sentences.append (fr_char_sentence)
    fr_char_sentences_shift.append (fr_char_sentence_shift)
    samples = samples + 1
    line = fp.readline()

# Get the length of the longest sentence
max_eng_sentence_length = max_sentence_length (eng_char_sentences)
max_fr_sentence_length = max_sentence_length (fr_char_sentences)

print (len (eng_char_sentences), len (fr_char_sentences))
print (eng_char_sentences[10], fr_char_sentences[10], fr_char_sentences_shift[10])
print (eng_char_sentences[75], fr_char_sentences[75], fr_char_sentences_shift[75])
print (max_eng_sentence_length, max_fr_sentence_length)

10000 10000
['S', 't', 'o', 'p', '!'] ['\t', 'Ç', 'a', ' ', 's', 'u', 'f', 'f', 'i', 't', '\u202f', '!', '\n'] ['Ç', 'a', ' ', 's', 'u', 'f', 'f', 'i', 't', '\u202f', '!', '\n']
['A', 'w', 'e', 's', 'o', 'm', 'e', '!'] ['\t', 'F', 'a', 'n', 't', 'a', 's', 't', 'i', 'q', 'u', 'e', '\u202f', '!', '\n'] ['F', 'a', 'n', 't', 'a', 's', 't', 'i', 'q', 'u', 'e', '\u202f', '!', '\n']
16 59


In [0]:
def vocab (char_sentences):
  merged_chars = [char for char_sentence in char_sentences for char in char_sentence]
  vocab_chars = sorted (set (merged_chars))
  vocab_size = len (vocab_chars)
  return (vocab_chars, vocab_size)

# Get the set of characters in the vocab
eng_vocab_chars, eng_vocab_size = vocab (eng_char_sentences)
fr_vocab_chars, fr_vocab_size = vocab (fr_char_sentences)

eng_vocab_size, fr_vocab_size

(70, 93)

In [0]:
# Dictionary to map from char to char_index in the vocab
eng_vocab_dict = {c:i for i, c in enumerate (eng_vocab_chars)}
fr_vocab_dict = {c:i for i, c in enumerate (fr_vocab_chars)}

# Char -> index
def char2idx (vocab_dict, c):
  return (vocab_dict [c])

# Index -> char
def idx2char (vocab_chars, i):
  return (vocab_chars[i])
  
print (char2idx (eng_vocab_dict, 'G'))
print (idx2char (eng_vocab_chars, 7))

26
-


In [0]:
encoder_input_data = np.zeros((num_samples, max_eng_sentence_length, eng_vocab_size), dtype='float32')
decoder_input_data = np.zeros((num_samples, max_fr_sentence_length, fr_vocab_size), dtype='float32')
decoder_target_data = np.zeros((num_samples, max_fr_sentence_length, fr_vocab_size), dtype='float32')

def char_indices (data, char_sentences, vocab_dict):
  for i, char_sentence in enumerate (char_sentences):
    for j, char in enumerate (char_sentence):
      idx = char2idx (vocab_dict, char)
      #print (char_sentence, i, j, idx)
      data [i, j, idx] = 1.0

char_indices (encoder_input_data, eng_char_sentences, eng_vocab_dict)
char_indices (decoder_input_data, fr_char_sentences, fr_vocab_dict)
char_indices (decoder_target_data, fr_char_sentences_shift, fr_vocab_dict)

In [0]:
num_encoder_tokens = eng_vocab_size
num_decoder_tokens = fr_vocab_size

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

In [0]:
max_encoder_seq_length = max_eng_sentence_length
max_decoder_seq_length = max_fr_sentence_length
input_token_index = eng_vocab_dict
target_token_index = fr_vocab_dict
input_texts = eng_char_sentences

# Next: inference mode (sampling).
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        #print ('Target seq = ', target_seq.shape)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # KD - the original logic was to always use a target sequence
        # of length 1, which we update with the last sampled token. We
        # also feed the states_value back as the initial_state for the
        # next iteration
        #
        # The alternate logic which I tried was to keep extending the target
        # sequence in each iteration, so that the last sampled token is appended
        # to the previous target sequence. If we do that we don't change the
        # initial_state value at all, and continue to use the earlier value
        #
        # Both alternates are giving the same results. But I should experiment
        # some more, try out other unseen input sequences and see how both
        # alternates perform
        kdAlternate = True
        if (kdAlternate):
          b = np.zeros((1, target_seq.shape[1] + 1, num_decoder_tokens))
          b[:,:-1] = target_seq
          b[0, -1, sampled_token_index] = 1.
          target_seq = b
        else:
          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.

          # Update states
          states_value = [h, c]

    return decoded_sentence


for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['G', 'o', '.']
Decoded sentence: Va !

-
Input sentence: ['H', 'i', '.']
Decoded sentence: Salut !

-
Input sentence: ['H', 'i', '.']
Decoded sentence: Salut !

-
Input sentence: ['R', 'u', 'n', '!']
Decoded sentence: Cours !

-
Input sentence: ['R', 'u', 'n', '!']
Decoded sentence: Cours !

-
Input sentence: ['W', 'h', 'o', '?']
Decoded sentence: Qui ?

-
Input sentence: ['W', 'o', 'w', '!']
Decoded sentence: Ça alors !

-
Input sentence: ['F', 'i', 'r', 'e', '!']
Decoded sentence: Au feu !

-
Input sentence: ['H', 'e', 'l', 'p', '!']
Decoded sentence: À l'aide !

-
Input sentence: ['J', 'u', 'm', 'p', '.']
Decoded sentence: Saute.

-
Input sentence: ['S', 't', 'o', 'p', '!']
Decoded sentence: Arrête-toi !

-
Input sentence: ['S', 't', 'o', 'p', '!']
Decoded sentence: Arrête-toi !

-
Input sentence: ['S', 't', 'o', 'p', '!']
Decoded sentence: Arrête-toi !

-
Input sentence: ['W', 'a', 'i', 't', '!']
Decoded sentence: Attendez !

-
Input sentence: ['W', 'a', 'i', 't'

In [0]:
# TEMP - just for trial experimentation, can be removed
N = 6
a = np.random.rand(1, 1, N)
b = np.zeros((1, a.shape[1] + 1, N))
b[:,:-1] = a
a, b, a.shape, b.shape

(array([[[0.19266532, 0.60973315, 0.18075967, 0.07692063, 0.48325006,
          0.5832694 ]]]),
 array([[[0.19266532, 0.60973315, 0.18075967, 0.07692063, 0.48325006,
          0.5832694 ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ]]]),
 (1, 1, 6),
 (1, 2, 6))

In [0]:
# Trying to understand shapes - move this into an earlier cell
print (encoder_inputs, '\n', encoder_outputs, '\n', state_h, '\n', state_c)
print (decoder_inputs, '\n', decoder_outputs, '\n', decoder_states_inputs)

Tensor("input_3:0", shape=(?, ?, 70), dtype=float32) 
 Tensor("lstm_3/TensorArrayReadV3:0", shape=(?, 256), dtype=float32) 
 Tensor("lstm_4_4/while/Exit_2:0", shape=(?, 256), dtype=float32) 
 Tensor("lstm_4_4/while/Exit_3:0", shape=(?, 256), dtype=float32)
Tensor("input_4:0", shape=(?, ?, 93), dtype=float32) 
 Tensor("dense_2_4/truediv:0", shape=(?, ?, 93), dtype=float32) 
 [<tf.Tensor 'input_11:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'input_12:0' shape=(?, 256) dtype=float32>]
